# U-Net

## Imports

In [6]:
from tensorflow.keras import layers, optimizers, backend, Model
from tensorflow.keras.applications import EfficientNetB4, EfficientNetV2S, ResNet50V2, DenseNet201

## Full model

<img src='images/unet.png' width='600'/>

In [14]:
# U-Net
def unet(input_shape=(512, 512, 3)):
    """
    Creates a neural network using the U-Net architecture.
    Args:
        input_shape: The size of the input image.
    Returns:
        A U-Net model.
    """
    # Encoder Part
    # Layer 1
    inputs = layers.Input(input_shape)
    inputs_rescaled = layers.Lambda(lambda x: x / 255)(inputs)  # Rescale input pixel values to floating point values
    c1 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs_rescaled)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    c1 = layers.BatchNormalization()(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    # Layer 2
    c2 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    c2 = layers.BatchNormalization()(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Layer 3
    c3 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    c3 = layers.BatchNormalization()(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Layer 4
    c4 = layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    c4 = layers.BatchNormalization()(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Layer 5
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = layers.BatchNormalization()(c5)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    c5 = layers.BatchNormalization()(c5)

    # Decoder Part
    # Layer 6
    u6 = layers.Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = layers.BatchNormalization()(c6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
    c6 = layers.BatchNormalization()(c6)

    # Layer 7
    u7 = layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = layers.BatchNormalization()(c7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
    c7 = layers.BatchNormalization()(c7)

    # Layer 8
    u8 = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = layers.BatchNormalization()(c8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
    c8 = layers.BatchNormalization()(c8)

    # Layer 9
    u9 = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1], axis=3)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = layers.BatchNormalization()(c9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    c9 = layers.BatchNormalization()(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    # Compiling model
    model = Model(inputs=[inputs], outputs=[outputs], name='UNet')
    return model

In [15]:
model = unet()

In [16]:
model.summary()

Model: "UNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 lambda_3 (Lambda)              (None, 512, 512, 3)  0           ['input_5[0][0]']                
                                                                                                  
 conv2d_48 (Conv2D)             (None, 512, 512, 64  1792        ['lambda_3[0][0]']               
                                )                                                                 
                                                                                               

                                                                                                  
 conv2d_58 (Conv2D)             (None, 64, 64, 512)  4719104     ['concatenate_9[0][0]']          
                                                                                                  
 batch_normalization_56 (BatchN  (None, 64, 64, 512)  2048       ['conv2d_58[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_59 (Conv2D)             (None, 64, 64, 512)  2359808     ['batch_normalization_56[0][0]'] 
                                                                                                  
 batch_normalization_57 (BatchN  (None, 64, 64, 512)  2048       ['conv2d_59[0][0]']              
 ormalization)                                                                                    
          

## Block functions

In [3]:
def conv_block(input, num_filters):
    """
    Creates a block consisting of two convolutional layers.
    Args:
        input: The output of the previous up-convolution.
        num_filters: Number of filters to be used in the convolution.
    Returns:
        The result of the final calculation of the current layer.
    """
    x = layers.Conv2D(num_filters, 3, activation='relu', kernel_initializer='he_normal',  padding='same')(input)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(num_filters, 3, activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = layers.BatchNormalization()(x)
    return x


def decoder_block(input, skip_features, num_filters):
    """
    Create a decoder block used in the decoder part of the network.
    Args:
        input: The output of the previous layer.
        skip_features: The features of the same size from the encoder part of the network.
        num_filters: Number of filters to be used in the convolution.
    Returns:
        The result of the final calculation of the current layer.
    """
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding='same')(input)
    x = layers.concatenate([x, skip_features])
    x = conv_block(x, num_filters)
    return x

## EfficientNetB4

In [4]:
# EfficientNetB4 UNet
def EfficientNetB4_unet(input_shape=(512, 512, 3), weight='imagenet'):
    """
    Creates a neural network using the U-Net architecture and EfficientNetB4 as backbone.
    Args:
        input_shape: The size of the input image.
        weight: Pre-trained weights.
    Returns:
        A U-Net model using EfficientNetB4 as backbone.
    """
    # Input
    inputs = layers.Input(input_shape)

    # Loading pre trained model
    EffNetB4 = EfficientNetB4(include_top=False, weights=weight, input_tensor=inputs)

    # Encoder
    s1 = EffNetB4.get_layer('rescaling').output  # 512 x 512
    s2 = EffNetB4.get_layer('block2a_expand_activation').output  # 256 x 256
    s3 = EffNetB4.get_layer('block3a_expand_activation').output  # 128 x 128
    s4 = EffNetB4.get_layer('block4a_expand_activation').output  # 64 x 64

    # Bottleneck
    b1 = EffNetB4.get_layer('block6a_expand_activation').output  # 32 x 32

    # Decoder
    d1 = decoder_block(b1, s4, 1024)  # 64 x 64
    d2 = decoder_block(d1, s3, 512)    # 128 x 128
    d3 = decoder_block(d2, s2, 256)   # 256 x 256
    d4 = decoder_block(d3, s1, 128)   # 512 x 512

    # Output
    outputs = layers.Conv2D(1, 1, padding='same', activation='sigmoid')(d4)

    model = Model(inputs, outputs, name='EfficientNetB4_U-Net')
    return model

In [5]:
model = EfficientNetB4_unet()

Model: "efficientnetb4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 512, 512, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 512, 512, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 512, 512, 3)  0           ['normalization[0][0

 block2a_expand_conv (Conv2D)   (None, 256, 256, 14  3456        ['block1b_add[0][0]']            
                                4)                                                                
                                                                                                  
 block2a_expand_bn (BatchNormal  (None, 256, 256, 14  576        ['block2a_expand_conv[0][0]']    
 ization)                       4)                                                                
                                                                                                  
 block2a_expand_activation (Act  (None, 256, 256, 14  0          ['block2a_expand_bn[0][0]']      
 ivation)                       4)                                                                
                                                                                                  
 block2a_dwconv_pad (ZeroPaddin  (None, 257, 257, 14  0          ['block2a_expand_activation[0][0]
 g2D)     

                                                                                                  
 block2c_expand_bn (BatchNormal  (None, 128, 128, 19  768        ['block2c_expand_conv[0][0]']    
 ization)                       2)                                                                
                                                                                                  
 block2c_expand_activation (Act  (None, 128, 128, 19  0          ['block2c_expand_bn[0][0]']      
 ivation)                       2)                                                                
                                                                                                  
 block2c_dwconv (DepthwiseConv2  (None, 128, 128, 19  1728       ['block2c_expand_activation[0][0]
 D)                             2)                               ']                               
                                                                                                  
 block2c_b

                                2)                                                                
                                                                                                  
 block3a_expand_bn (BatchNormal  (None, 128, 128, 19  768        ['block3a_expand_conv[0][0]']    
 ization)                       2)                                                                
                                                                                                  
 block3a_expand_activation (Act  (None, 128, 128, 19  0          ['block3a_expand_bn[0][0]']      
 ivation)                       2)                                                                
                                                                                                  
 block3a_dwconv_pad (ZeroPaddin  (None, 131, 131, 19  0          ['block3a_expand_activation[0][0]
 g2D)                           2)                               ']                               
          

                                                                                                  
 block3c_dwconv (DepthwiseConv2  (None, 64, 64, 336)  8400       ['block3c_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3c_bn (BatchNormalization  (None, 64, 64, 336)  1344       ['block3c_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block3c_activation (Activation  (None, 64, 64, 336)  0          ['block3c_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block3c_s

...

 block6b_bn (BatchNormalization  (None, 16, 16, 1632  6528       ['block6b_dwconv[0][0]']         
 )                              )                                                                 
                                                                                                  
 block6b_activation (Activation  (None, 16, 16, 1632  0          ['block6b_bn[0][0]']             
 )                              )                                                                 
                                                                                                  
 block6b_se_squeeze (GlobalAver  (None, 1632)        0           ['block6b_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block6b_se_reshape (Reshape)   (None, 1, 1, 1632)   0           ['block6b_se_squeeze[0][0]']     
          

 block6d_activation (Activation  (None, 16, 16, 1632  0          ['block6d_bn[0][0]']             
 )                              )                                                                 
                                                                                                  
 block6d_se_squeeze (GlobalAver  (None, 1632)        0           ['block6d_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block6d_se_reshape (Reshape)   (None, 1, 1, 1632)   0           ['block6d_se_squeeze[0][0]']     
                                                                                                  
 block6d_se_reduce (Conv2D)     (None, 1, 1, 68)     111044      ['block6d_se_reshape[0][0]']     
                                                                                                  
 block6d_s

 block6f_se_squeeze (GlobalAver  (None, 1632)        0           ['block6f_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block6f_se_reshape (Reshape)   (None, 1, 1, 1632)   0           ['block6f_se_squeeze[0][0]']     
                                                                                                  
 block6f_se_reduce (Conv2D)     (None, 1, 1, 68)     111044      ['block6f_se_reshape[0][0]']     
                                                                                                  
 block6f_se_expand (Conv2D)     (None, 1, 1, 1632)   112608      ['block6f_se_reduce[0][0]']      
                                                                                                  
 block6f_se_excite (Multiply)   (None, 16, 16, 1632  0           ['block6f_activation[0][0]',     
          

 block6h_se_reshape (Reshape)   (None, 1, 1, 1632)   0           ['block6h_se_squeeze[0][0]']     
                                                                                                  
 block6h_se_reduce (Conv2D)     (None, 1, 1, 68)     111044      ['block6h_se_reshape[0][0]']     
                                                                                                  
 block6h_se_expand (Conv2D)     (None, 1, 1, 1632)   112608      ['block6h_se_reduce[0][0]']      
                                                                                                  
 block6h_se_excite (Multiply)   (None, 16, 16, 1632  0           ['block6h_activation[0][0]',     
                                )                                 'block6h_se_expand[0][0]']      
                                                                                                  
 block6h_project_conv (Conv2D)  (None, 16, 16, 272)  443904      ['block6h_se_excite[0][0]']      
          

                                                                                                  
 block7b_project_conv (Conv2D)  (None, 16, 16, 448)  1204224     ['block7b_se_excite[0][0]']      
                                                                                                  
 block7b_project_bn (BatchNorma  (None, 16, 16, 448)  1792       ['block7b_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block7b_drop (Dropout)         (None, 16, 16, 448)  0           ['block7b_project_bn[0][0]']     
                                                                                                  
 block7b_add (Add)              (None, 16, 16, 448)  0           ['block7b_drop[0][0]',           
                                                                  'block7a_project_bn[0][0]']     
          

In [5]:
model.summary()

Model: "EfficientNetB4_U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 512, 512, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 512, 512, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 512, 512, 3)  0           ['normalizatio

 block2a_expand_conv (Conv2D)   (None, 256, 256, 14  3456        ['block1b_add[0][0]']            
                                4)                                                                
                                                                                                  
 block2a_expand_bn (BatchNormal  (None, 256, 256, 14  576        ['block2a_expand_conv[0][0]']    
 ization)                       4)                                                                
                                                                                                  
 block2a_expand_activation (Act  (None, 256, 256, 14  0          ['block2a_expand_bn[0][0]']      
 ivation)                       4)                                                                
                                                                                                  
 block2a_dwconv_pad (ZeroPaddin  (None, 257, 257, 14  0          ['block2a_expand_activation[0][0]
 g2D)     

                                                                                                  
 block2c_expand_bn (BatchNormal  (None, 128, 128, 19  768        ['block2c_expand_conv[0][0]']    
 ization)                       2)                                                                
                                                                                                  
 block2c_expand_activation (Act  (None, 128, 128, 19  0          ['block2c_expand_bn[0][0]']      
 ivation)                       2)                                                                
                                                                                                  
 block2c_dwconv (DepthwiseConv2  (None, 128, 128, 19  1728       ['block2c_expand_activation[0][0]
 D)                             2)                               ']                               
                                                                                                  
 block2c_b

                                2)                                                                
                                                                                                  
 block3a_expand_bn (BatchNormal  (None, 128, 128, 19  768        ['block3a_expand_conv[0][0]']    
 ization)                       2)                                                                
                                                                                                  
 block3a_expand_activation (Act  (None, 128, 128, 19  0          ['block3a_expand_bn[0][0]']      
 ivation)                       2)                                                                
                                                                                                  
 block3a_dwconv_pad (ZeroPaddin  (None, 131, 131, 19  0          ['block3a_expand_activation[0][0]
 g2D)                           2)                               ']                               
          

                                                                                                  
 block3c_dwconv (DepthwiseConv2  (None, 64, 64, 336)  8400       ['block3c_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3c_bn (BatchNormalization  (None, 64, 64, 336)  1344       ['block3c_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block3c_activation (Activation  (None, 64, 64, 336)  0          ['block3c_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block3c_s

...

 block4e_se_expand (Conv2D)     (None, 1, 1, 672)    19488       ['block4e_se_reduce[0][0]']      
                                                                                                  
 block4e_se_excite (Multiply)   (None, 32, 32, 672)  0           ['block4e_activation[0][0]',     
                                                                  'block4e_se_expand[0][0]']      
                                                                                                  
 block4e_project_conv (Conv2D)  (None, 32, 32, 112)  75264       ['block4e_se_excite[0][0]']      
                                                                                                  
 block4e_project_bn (BatchNorma  (None, 32, 32, 112)  448        ['block4e_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block4e_d

 block5a_project_conv (Conv2D)  (None, 32, 32, 160)  107520      ['block5a_se_excite[0][0]']      
                                                                                                  
 block5a_project_bn (BatchNorma  (None, 32, 32, 160)  640        ['block5a_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block5b_expand_conv (Conv2D)   (None, 32, 32, 960)  153600      ['block5a_project_bn[0][0]']     
                                                                                                  
 block5b_expand_bn (BatchNormal  (None, 32, 32, 960)  3840       ['block5b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block5b_e

 block5d_expand_conv (Conv2D)   (None, 32, 32, 960)  153600      ['block5c_add[0][0]']            
                                                                                                  
 block5d_expand_bn (BatchNormal  (None, 32, 32, 960)  3840       ['block5d_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block5d_expand_activation (Act  (None, 32, 32, 960)  0          ['block5d_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block5d_dwconv (DepthwiseConv2  (None, 32, 32, 960)  24000      ['block5d_expand_activation[0][0]
 D)                                                              ']                               
          

 block5f_expand_activation (Act  (None, 32, 32, 960)  0          ['block5f_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block5f_dwconv (DepthwiseConv2  (None, 32, 32, 960)  24000      ['block5f_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block5f_bn (BatchNormalization  (None, 32, 32, 960)  3840       ['block5f_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block5f_activation (Activation  (None, 32, 32, 960)  0          ['block5f_bn[0][0]']             
 )        

 concatenate_2 (Concatenate)    (None, 256, 256, 40  0           ['conv2d_transpose_2[0][0]',     
                                0)                                'block2a_expand_activation[0][0]
                                                                 ']                               
                                                                                                  
 conv2d_4 (Conv2D)              (None, 256, 256, 25  921856      ['concatenate_2[0][0]']          
                                6)                                                                
                                                                                                  
 batch_normalization_4 (BatchNo  (None, 256, 256, 25  1024       ['conv2d_4[0][0]']               
 rmalization)                   6)                                                                
                                                                                                  
 conv2d_5 

## ResNet50V2

In [3]:
def ResNet50V2_unet(input_shape=(512, 512, 3), weight='imagenet'):
    """
    Creates a neural network using the U-Net architecture and ResNet50V2 as backbone.
    Args:
        input_shape: The size of the input image.
        weight: Pre-trained weights.
    Returns:
        A U-Net model using EfficientNetB4 as backbone.
    """
    # Input
    inputs = layers.Input(input_shape)

    # Loading pre trained model
    ResNet50 = ResNet50V2(include_top=False, weights=weight, input_tensor=inputs)

    # Encoder
    s1 = ResNet50.get_layer('input_1').output  # 512 x 512
    s2 = ResNet50.get_layer('conv1_conv').output  # 256 x 256
    s3 = ResNet50.get_layer('conv2_block3_1_relu').output  # 128 x 128
    s4 = ResNet50.get_layer('conv3_block4_1_relu').output  # 64 x 64

    # Bottleneck
    b1 = ResNet50.get_layer('conv4_block6_1_relu').output  # 32 x 32

    # Decoder
    d1 = decoder_block(b1, s4, 1024)  # 64 x 64
    d2 = decoder_block(d1, s3, 512)    # 128 x 128
    d3 = decoder_block(d2, s2, 256)   # 256 x 256
    d4 = decoder_block(d3, s1, 128)   # 512 x 512

    # Output
    outputs = layers.Conv2D(1, 1, padding='same', activation='sigmoid')(d4)

    model = Model(inputs, outputs, name='ResNet50V2_U-Net')
    return model

In [4]:
model = ResNet50V2_unet()

In [5]:
model.summary()

Model: "ResNet50V2_U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                   

 conv2_block2_out (Add)         (None, 128, 128, 25  0           ['conv2_block1_out[0][0]',       
                                6)                                'conv2_block2_3_conv[0][0]']    
                                                                                                  
 conv2_block3_preact_bn (BatchN  (None, 128, 128, 25  1024       ['conv2_block2_out[0][0]']       
 ormalization)                  6)                                                                
                                                                                                  
 conv2_block3_preact_relu (Acti  (None, 128, 128, 25  0          ['conv2_block3_preact_bn[0][0]'] 
 vation)                        6)                                                                
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 128, 128, 64  16384       ['conv2_block3_preact_relu[0][0]'
          

 n)                                                                                               
                                                                                                  
 conv3_block2_2_pad (ZeroPaddin  (None, 66, 66, 128)  0          ['conv3_block2_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block2_2_conv (Conv2D)   (None, 64, 64, 128)  147456      ['conv3_block2_2_pad[0][0]']     
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, 64, 64, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_blo

 ormalization)                                                                                    
                                                                                                  
 conv4_block1_preact_relu (Acti  (None, 32, 32, 512)  0          ['conv4_block1_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv4_block1_1_conv (Conv2D)   (None, 32, 32, 256)  131072      ['conv4_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_block1_1_bn (BatchNormal  (None, 32, 32, 256)  1024       ['conv4_block1_1_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 conv4_block3_2_conv (Conv2D)   (None, 32, 32, 256)  589824      ['conv4_block3_2_pad[0][0]']     
                                                                                                  
 conv4_block3_2_bn (BatchNormal  (None, 32, 32, 256)  1024       ['conv4_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block3_2_relu (Activatio  (None, 32, 32, 256)  0          ['conv4_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block3_3_conv (Conv2D)   (None, 32, 32, 1024  263168      ['conv4_block3_2_relu[0][0]']    
          

 vation)                        )                                                                 
                                                                                                  
 conv4_block6_1_conv (Conv2D)   (None, 32, 32, 256)  262144      ['conv4_block6_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_block6_1_bn (BatchNormal  (None, 32, 32, 256)  1024       ['conv4_block6_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block6_1_relu (Activatio  (None, 32, 32, 256)  0          ['conv4_block6_1_bn[0][0]']      
 n)                                                                                               
          

 conv2d_transpose_3 (Conv2DTran  (None, 512, 512, 12  131200     ['activation_5[0][0]']           
 spose)                         8)                                                                
                                                                                                  
 concatenate_3 (Concatenate)    (None, 512, 512, 13  0           ['conv2d_transpose_3[0][0]',     
                                1)                                'input_1[0][0]']                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 512, 512, 12  151040      ['concatenate_3[0][0]']          
                                8)                                                                
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 512, 512, 12  512        ['conv2d_6[0][0]']               
 rmalizati